# Performance Results

The goal here is to compare the performance results with different contexts:
- Baseline (No Context)
- RAG all Wikipedia (FAISS)
- RAG selected samples
- Datamodels


## Summary

- [Load Data]()
- [Create FAISS]()
- [Create different contexts]()
- [Run Inferences]()
- [Compare Results]()

In [1]:
import polars as pl
import torch
import os
import random
import numpy as np


os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed = 42

torch.backends.cudnn.enabled = False
# NumPy
np.random.seed(seed)
random.seed(seed)
# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Number of GPUs available: 1
GPU 0: NVIDIA RTX A5000


## Load Data

In [2]:
## Questions

questions = []
for i in range(5):
    questions.append(pl.read_csv(f"question_{i}_datamodels/test_set.csv"))

questions = pl.concat(questions)
questions.select("question").to_numpy().flatten()

array(['when did the who perform at the super bowl',
       'why was there so much interest in cuba both before and after the civil war',
       'who developed the first periodic table with 8 columns',
       'the organization of the formal elements in an art work',
       'where did the french king live before versailles'], dtype=object)

In [3]:
### Train samples
train_sets = []
for i in range(5):
    train_sets.append(pl.read_csv(f"question_{i}_datamodels/train_set.csv").with_columns(pl.lit(f"question_{i}").alias("question_id")))
train = pl.concat(train_sets)

### Load weights
weights_set = []
for i in range(5):
    weights = torch.load(f"question_{i}_datamodels/models/regression_question_{i}/weights.pt").to("cpu")
    weights_set.append(weights)

## add weights to sample training
list_weights = []
for set in weights_set:
    for _w in set[0]:
        list_weights.append(float(_w.item()))


train = train.with_columns(
    pl.Series("weights", list_weights),
    pl.when(pl.col("idx_right").is_not_null())
        .then(True)
        .otherwise(False)
        .alias("is_from_same_page")

)
train.head()

/tmp/ipykernel_2849775/2729841633.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(f"question_{i}_datamodels/models/regression_question_{i}/weights.

idx,text,title,idx_right,question_id,weights,is_from_same_page
i64,str,str,i64,str,f64,bool
0,"""Date : Feb 5 , 2017 Location :…","""List of Super Bowl halftime sh…",20970820,"""question_0""",0.038918,true
1,"""Date : Feb 4 , 2018 Location :…","""List of Super Bowl halftime sh…",20970820,"""question_0""",-0.15716,true
2,"""Date : Feb 3 , 2013 Location :…","""List of Super Bowl halftime sh…",20970820,"""question_0""",-0.050358,true
3,"""The NFL does not pay the halft…","""List of Super Bowl halftime sh…",20970820,"""question_0""",-0.164697,true
4,"""LI Main article : Super Bowl L…","""List of Super Bowl halftime sh…",20970820,"""question_0""",-0.113502,true


In [4]:
## Load Wiki
WIKI_PATH = "../../data/wiki_dump2018_nq_open/processed/wiki.feather"
wiki = pl.read_ipc(WIKI_PATH).with_row_index("idx")
wiki.head()

idx,text,title
u32,str,str
0,"""Aaron Aaron ( or ; ""Ahärôn"") i…","""Aaron"""
1,"""God at Sinai granted Aaron the…","""Aaron"""
2,"""his rod turn into a snake. The…","""Aaron"""
3,"""however, Aaron and Hur remaine…","""Aaron"""
4,"""Aaron and his sons to the prie…","""Aaron"""


In [5]:
target = []
random = []
for i in range(5):
    target_i = pl.read_ipc(f"question_{i}_datamodels/target.feather").with_columns(pl.lit(f"question_{i}").alias("question_id"))
    random_i = pl.read_ipc(f"question_{i}_datamodels/random.feather").with_columns(pl.lit(f"question_{i}").alias("question_id"))
    target.append(target_i)
    random.append(random_i)

target = pl.concat(target)
random = pl.concat(random)

## Create FAISS

In [6]:
from FlagEmbedding import FlagModel
import faiss
from tqdm import tqdm  

EMBERDDER_PATH = "../../models/llms/bge-base-en-v1.5"
embedder = FlagModel(EMBERDDER_PATH, devices=["cuda:0"], use_fp16=True)



/home/caio.rhoden/miniconda3/envs/nq/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
index = faiss.IndexScalarQuantizer(768, 8)
# index = faiss.read_index("wiki.index")

In [10]:

total_size = len(wiki)
batch_size = 80000
torch.backends.cuda.enable_cudnn_sdp(False)

for start in range(80000, total_size, batch_size):
    
    end = min(start + batch_size, total_size)
    print(f"End: {end}")
    
    batch_texts = wiki[start:end].select("text").to_numpy().flatten().tolist()
    
    # Encode the current batch
    batch_embeddings = embedder.encode(
        batch_texts,
        convert_to_numpy=True,
    )
    
    # Add to index
    index.add(batch_embeddings.astype('float32'))
    faiss.write_index(index, "wiki.index")

    
    # Optional: Clear memory if needed
    del batch_texts, batch_embeddings
    break

End: 160000


Inference Embeddings: 100%|██████████| 313/313 [00:36<00:00,  8.66it/s]


In [52]:
questions[1]["question"].to_numpy().flatten()[0]

'why was there so much interest in cuba both before and after the civil war'

In [53]:
### Demo FAISS search
test_query = questions[1]["question"].to_numpy().flatten()[0]
query_embedding = embedder.encode(
    [test_query],
    convert_to_numpy=True,
)
query_embedding = query_embedding.astype('float32').reshape(1, -1)
distances, indices = index.search(query_embedding, 4)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
indices, distances

(array([[15107718, 11485636,   380206,   380167]]),
 array([[0.7561871 , 0.7198814 , 0.71146166, 0.7023028 ]], dtype=float32))

: 

: 

## Create Contexts

In [63]:
### FAISS context
faiss_context = {}

for i in tqdm(range(len(questions))):

    question = questions[i]["question"].to_numpy().flatten()[0]
    query_embedding = embedder.encode(
        [question],
        convert_to_numpy=True,
    )
    query_embedding = query_embedding.astype('float32').reshape(1, -1)
    distances, indices = index.search(query_embedding, 4)

    faiss_context[f"question_{i}"] = [wiki[int(j)]["text"].to_numpy().flatten()[0] for j in indices[0]]
    print(f"Scores: {distances}")

 20%|██        | 1/5 [00:05<00:20,  5.11s/it]

Scores: [[0.8165758  0.801557   0.7908786  0.78759444]]


 40%|████      | 2/5 [00:10<00:16,  5.46s/it]

Scores: [[0.7561871  0.7198814  0.71146166 0.7023028 ]]


 60%|██████    | 3/5 [00:16<00:11,  5.61s/it]

Scores: [[0.8041481  0.8028399  0.78939456 0.78801113]]


 60%|██████    | 3/5 [00:22<00:14,  7.46s/it]


IndexError: index 21409643 is out of bounds for DataFrame of height 21035236

In [ ]:
datamodels_general_context = {}
for i in tqdm(range(len(questions))):
    weights = train.filter(pl.col("question_id") == f"question_{i}").select("").to_numpy().flatten()[0]
    
    

: 